In [1]:
from __future__ import print_function
import tensorflow as tf 

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
print (mnist.train.images.shape)

(55000, 784)


In [4]:
#Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

#Network Parameters
num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

#tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [5]:
#Define Weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
def RNN(x, weights, biases):
    #print (type(x))
    x = tf.unstack(x, timesteps, 1)
    #print (type(x))
    #print (x.get_shape())
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias = 1.0)#, reuse = True)
    
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [9]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

#Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
logits = logits, labels = Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)

#Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [17]:
import numpy as np 
with tf.Session() as sess:
    sess.run(init)
    for step in xrange(1,1000 + 1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        #Back prop
        sess.run(train_op, feed_dict={X: batch_x, Y:batch_y})
        if step % 100 == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={
                X:batch_x, Y:batch_y
            })
            print ("step", step, "loss", loss, "Acc: ", acc)
    print ("Optimization Finished")
    
    

step 1 loss 2.57476 Acc:  0.09375
step 100 loss 2.2126 Acc:  0.164062
step 200 loss 2.09828 Acc:  0.257812
step 300 loss 2.0539 Acc:  0.320312
step 400 loss 1.98228 Acc:  0.375
step 500 loss 1.82161 Acc:  0.421875
step 600 loss 1.84323 Acc:  0.359375
step 700 loss 1.66774 Acc:  0.476562
step 800 loss 1.67108 Acc:  0.492188
step 900 loss 1.66978 Acc:  0.492188
step 1000 loss 1.4698 Acc:  0.578125
Optimization Finished
